In [31]:
import datetime as dt
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join

In [32]:
#the month and day that has previous day cci <= -100 and today closed with nice green candle
y = '2021'
m = '02'
d = '04'
before_date = y + '-' + m + '-' + d
print(before_date)

2021-02-04


In [33]:
def indicators_calculation(df, before_date):
    
    #df is the filtered df to inspect
    df = df[(df['Date'] <= before_date)]
    
    #SMAs
    df['50SMA'] = df['Close'].rolling(window=50).mean().round(2)
    df['100SMA'] = df['Close'].rolling(window=100).mean().round(2)
    df['20 Volume'] = df['Volume'].rolling(window=20).mean().round(2)

    #5CCI
    #typical price used for cci calculation
    df['TP'] = (df['High'] + df['Low'] + df['Close']) / 3
    #mean deviation formula used in cci calculation
    MD = lambda x : np.mean(np.abs(x-np.mean(x)))
    #cci calculation
    df['CCI'] = ((df['TP'] - df['TP'].rolling(5).mean()) / (0.015 * df['TP'].rolling(5).apply(MD, True))).round(2) 

    df['A'] = np.where((df['Low'] > df['50SMA']), True, False)
    df['B'] = np.where((df['Low'] > df['100SMA']), True, False)
    df['Uptrend'] = np.where((df['50SMA'] > df['100SMA']), True, False)
    df['NON_PENNY'] = np.where((df['Low'] > 5), True, False)
    df['Liquid'] = np.where((df['20 Volume'] > 200000), True, False)

    return df


def indicator_condition(df_ticker_info):
    df_check = df_ticker_info[-50:]
    ratio = (df_check['Close'].iloc[-1] - df_check['Low'].iloc[-1]) / (df_check['High'].iloc[-1] - df_check['Low'].iloc[-1])
    if (df_check['CCI'].iloc[-2] <= -100) and (df_check['Close'].iloc[-1] > df_check['Open'].iloc[-1]) and (df_check['Close'].iloc[-1] > df_check['Close'].iloc[-2]) and (df_check['Close'].iloc[-1] > df_check['Open'].iloc[-2]) and (ratio >= 0.7)  and df_check['A'].all() and df_check['B'].all() and df_check['NON_PENNY'].all() and df_check['Liquid'].all() and df_check['Uptrend'].all():
        return True

In [34]:
path = r'C:\Users\michael\Desktop\trading materials\us market\us_tickers_history'
ticker_files = [f for f in listdir(path) if isfile(join(path, f))]
ticker_files_array = np.array(ticker_files)
ticker_files_split = ticker_files_array

n = 0
wanted_list = []
ratio_list = []
for i in ticker_files_split:
    df_ticker = pd.read_csv(join(path, i))
    df_ticker_info = indicators_calculation(df_ticker, before_date)
    if (indicator_condition(df_ticker_info)):
        wanted_list.append(i[:-4])
        ratio = (df_ticker_info['Close'].iloc[-1] - df_ticker_info['Low'].iloc[-1]) / (df_ticker_info['High'].iloc[-1] - df_ticker_info['Low'].iloc[-1])
        ratio_list.append(ratio)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

In [35]:
print(wanted_list)

['SBGI', 'VCRA']


In [36]:
path2 = r'C:\Users\michael\Desktop\trading materials\us market\uptrend_swing\compiled'
df = pd.DataFrame(list(zip(wanted_list, ratio_list)), columns =['Name', 'Ratio'])
filename = before_date + '.xlsx'
df.to_excel(join(path2, filename), index=False)